In [2]:
import random
random.seed(42)
from quotes_5k_dataloader import QuoteDB
import re
from pathlib import Path
%load_ext autoreload
%autoreload 2

D:\Playground\coursera_honors\quotes_5k_dataloader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from preprocess import contractions
# clean dataset
def clean_dataset(text):
    # To lowercase
    text = text.lower()
    # replacing contractions
    text = contractions[text.lower()] if text.lower() in contractions.keys() else text
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Insert space between words and punctuation marks
    text = re.sub( r'([a-zA-Z])([,.!])', r'\1 \2', text)
    text = re.sub( r'([,.!])([a-zA-Z])', r'\1 \2', text)
    return text

In [4]:
quote_db = QuoteDB("data/quotesdrivedb.csv", preprocessor=clean_dataset)


Skipped 0 quotes


In [12]:
def split_save_dataset(tags):
    quotes = []
    for tag in tags:
        _quotes = quote_db.get_persona_corpus(tag)
        quotes += _quotes
    tag = "_".join([tag.lower() for tag in tags])
    val_split = 0.01
    test_split = 0.01
    val_length = int(len(quotes)*val_split)
    test_length = int(len(quotes)*test_split)
    random.shuffle(quotes)
    quotes_test = quotes[-test_length:]
    quotes = quotes[:-test_length]
    quotes_val = quotes[-val_length:]
    quotes = quotes[:-val_length]

    with open(Path().cwd()/"data"/"quotesdb"/f"{tag}_quotes.train.txt", 'w', encoding='utf-8') as filehandle:
        for quote in quotes:
            filehandle.write('%s\n' % quote)
    with open(Path().cwd()/"data"/"quotesdb"/f"{tag}_quotes.val.txt", 'w', encoding='utf-8') as filehandle:
        for quote in quotes_val:
            filehandle.write('%s\n' % quote)

    with open(Path().cwd()/"data"/"quotesdb"/f"{tag}_quotes.test.txt", 'w', encoding='utf-8') as filehandle:
        for quote in quotes_test:
            filehandle.write('%s\n' % quote)

In [13]:
split_save_dataset(['FUNNY'])

In [14]:
split_save_dataset(['MOTIVATIONAL'])

In [15]:
split_save_dataset(['SERIOUS'])

In [16]:
split_save_dataset(["FUNNY","MOTIVATIONAL",'SERIOUS'])